In [1]:
import pandas as pd
import pandas_datareader as pdr

### 1. In this notebook, I have tried to implement the mean reversion trading strategy
### 2. If the time series value is far from mean in the positive side, go short otherwise go long
